In [118]:
from __future__ import unicode_literals, print_function

import pandas as pd
import os
import plac
import random
import warnings
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding

In [119]:
data = []
with open("treino.txt", "r") as f:
    for line in f.readlines():
        data.append(line.replace('\n', ''))
dataset = pd.DataFrame({'corpus': list(set(data))})

In [120]:
dataset.head()

,corpus
0,DIRETORIA DE HABILITAÇÃO
1,"Portaria DH-386, de 14-1-2021"
2,realizada em 02-12-2020 encetada pelas Ordens ...
3,"Antonio Carlos Pereira dos Santos, CPF 018.247..."
4,mento.


In [217]:
# carrega modelo em portugues
nlp = spacy.load('pt_core_news_sm')

In [218]:
text = "\n".join(dataset["corpus"])

In [219]:
# teste com a lib re
import re

terms = [ # Montar como regex
    ("Estampador de Placa de Identificação Veicular", "CARGO"),
    ("Celso Silva Santos Auto Escola", "ORG"),
    ("Diretor Setorial de Veículos do Detran-SP", "CARGO"),
    ("Diretor de Habilitação do Departamento Estadual de Trânsito - Detran/SP", "CARGO"),
    ("Comercio de Placas Ltda", "ORG")
]

TRAIN_DATA = []
for row in dataset['corpus']:
    for term in terms:
        x = re.search(term[0], row)
        if x:
            TRAIN_DATA.append((row, {"entities": [(x.span()[0], x.span()[1], term[1])]}))
print(TRAIN_DATA)

[('Estampador de Placa de Identificação Veicular, nos termos da', {'entities': [(0, 45, 'CARGO')]}), ('pelo CFC “A”: Celso Silva Santos Auto Escola, nome fantasia', {'entities': [(14, 44, 'ORG')]}), ('desfavor do CFC “A” Celso Silva Santos Auto Escola, nome fan-', {'entities': [(20, 50, 'ORG')]}), ('O Diretor Setorial de Veículos do Detran-SP resolve:', {'entities': [(2, 43, 'CARGO')]}), ('15.900-053 como Estampador de Placa de Identificação Veicular,', {'entities': [(16, 61, 'CARGO')]}), ('Comercio de Placas Ltda – CNPJ 37.430.585/0005-13 estabele-', {'entities': [(0, 23, 'ORG')]}), ('15.900-023 como Estampador de Placa de Identificação Veicular,', {'entities': [(16, 61, 'CARGO')]}), ('14.540-000 como Estampador de Placa de Identificação Veicular,', {'entities': [(16, 61, 'CARGO')]}), ('122 como Estampador de Placa de Identificação Veicular, nos', {'entities': [(9, 54, 'CARGO')]})]


In [220]:
# TRAIN_DATA = [
#     ("Estampador de Placa de Identificação Veicular, nos termos da", {"entities": [(0, 45, 'CARGO')]}),
#     ("pelo CFC “A”: Celso Silva Santos Auto Escola, nome fantasia", {"entities": [(14, 44, 'ORG')]}),
#     ("Celso Silva Santos, CPF 348.409.338-20, por transgressões ao", {"entities": [(0, 18, 'PERSON')]}),
#     ("5.900-053 como Estampador de Placa de Identificação Veicular,", {"entities": [(15, 60, 'CARGO')]}),
# ]

In [221]:
# Carrega modelo atual na lib
ner=nlp.get_pipe("ner")

# Adding labels to the `ner`
for _, annotations in TRAIN_DATA:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])

In [222]:
# predição sem treinamento
doc = nlp("O Diretor Setorial de Veículos do Detran-SP resolve: Artigo 1o. Credenciar a pessoa jurídica Solange Aires de Oliveira Souza Eireli – CNPJ 36.647.867/0001-80 estabelecida na R Jose Moreira Leite, 206 - Jardim Planalto – Itai – SP – 18.730-122 como Estampador de Placa de Identificação Veicular, nos termos da Resolução Contran 780/2019.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Diretor Setorial de Veículos do Detran-SP', 'PER'), ('Artigo', 'LOC'), ('Solange Aires de Oliveira Souza Eireli', 'PER'), ('CNPJ', 'MISC'), ('R Jose Moreira Leite', 'MISC'), ('Jardim Planalto', 'LOC'), ('Itai', 'LOC'), ('SP', 'LOC'), ('Estampador de Placa de Identificação Veicular', 'MISC'), ('Resolução Contran 780/2019', 'MISC')]


In [223]:
# Disable pipeline components you dont need to change
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

In [224]:
# Import requirements
import random
from spacy.util import minibatch, compounding
from pathlib import Path

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):

  # Training for 300 iterations
  for iteration in range(300):

    # shuufling examples  before every iteration
    random.shuffle(TRAIN_DATA)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
        # print("Losses", losses)

In [225]:
# predição depois do treinamento
doc = nlp("O Diretor Setorial de Veículos do Detran-SP resolve: Artigo 1o. Credenciar a pessoa jurídica Solange Aires de Oliveira Souza Eireli – CNPJ 36.647.867/0001-80 estabelecida na R Jose Moreira Leite, 206 - Jardim Planalto – Itai – SP – 18.730-122 como Estampador de Placa de Identificação Veicular, nos termos da Resolução Contran 780/2019.")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Diretor Setorial de Veículos do Detran-SP resolve: Artigo 1o.', 'CARGO'), ('Solange Aires de Oliveira Souza Eireli', 'ORG'), ('Estampador de Placa de Identificação Veicular', 'CARGO')]
